### Lab: Building a Single AI Agent using Strands

This notebook creates a single AI agent using Strands that can guide a home buyer through a preapproval for a new home loan application.
The agent handles creating new home loans and checking the status of existing applications using custom tools.

Step 1: Install the prequisites. Ignore errors 

In [1]:
!pip install --upgrade -q -r ../src/requirements.txt

Step 2: import libraries

In [2]:
import os
import json
from datetime import datetime, timedelta
from strands import Agent, tool

Step 3: Define Tools

In [3]:
@tool
def get_mortgage_app_doc_status(customer_id: str = None):
    """Retrieves the list of required documents for a mortgage application"""
    return [
        {"type": "proof_of_income", "status": "COMPLETED"},
        {"type": "employment_information", "status": "MISSING"},
        {"type": "proof_of_assets", "status": "COMPLETED"},
        {"type": "credit_information", "status": "COMPLETED"}
    ]

@tool
def get_application_details(customer_id: str = None):
    """Retrieves details about a mortgage application"""
    return {
        "customer_id": customer_id or "123456",
        "application_id": "998776",
        "application_date": str(datetime.today() - timedelta(days=35)),
        "application_status": "IN_PROGRESS",
        "application_type": "NEW_MORTGAGE",
        "name": "Mithil"
    }

@tool
def create_customer_id():
    """Creates a new customer ID"""
    return "123456"

@tool
def create_loan_application(customer_id: str, name: str, age: int, annual_income: int, annual_expense: int):
    """Creates a new loan application"""
    print(f"Creating loan application for customer: {customer_id}")
    print(f"Customer name: {name}")
    print(f"Customer age: {age}")
    print(f"Customer annual income: {annual_income}")
    print(f"Customer annual expense: {annual_expense}")
    return f"Loan application created successfully for {name} (ID: {customer_id})"

Step 4 : Specify choice of foundational models

In [4]:
agent_foundation_models = [ 
    "us.anthropic.claude-3-5-sonnet-20241022-v2:0",
    "amazon.nova-lite-v1:0"
    ]

Step 5: Create the Strands Agent

In [5]:
mortgage_application_agent = Agent(
    model=agent_foundation_models[0],
    tools=[get_mortgage_app_doc_status, get_application_details, create_customer_id, create_loan_application],
    system_prompt="""You are a mortgage application agent that helps customers with new mortgage applications.
    
    Instructions:
    - Greet customers warmly before answering
    - First ask for their customer ID, if they don't have one, create a new one
    - Ask for name, age, annual income, and annual expense one question at a time
    - Once you have all information, create the loan application
    - Only discuss mortgage applications, not general mortgage topics
    - Never make up information you cannot retrieve from tools"""
)

print("Mortgage Application Agent created successfully!")

Mortgage Application Agent created successfully!


Step 6 : Test the Agent. Try the question "Hello, I'd like to apply for a new mortgage"

In [7]:

def chat_loop():
        """Run an interactive chat loop"""
        print("\nMCP Chatbot Started!")
        print("Type your queries or 'quit' to exit.")
        while True:
            try:
                query = input("\nQuery: ").strip()
                if query.lower() == 'quit':
                    break
                response = mortgage_application_agent(query)
                print("\n")
            except Exception as e:
                print(f"\nError: {str(e)}")
chat_loop()

# In the chat window that appears below, try the question
# "Hello, I'd like to apply for a new mortgage"


MCP Chatbot Started!
Type your queries or 'quit' to exit.



Query:  quit


Step 7: Save the Agent

In [8]:
import pickle

agent_config = {
    'tools': [get_mortgage_app_doc_status, get_application_details, create_customer_id, create_loan_application],
    'system_prompt': mortgage_application_agent.system_prompt,
    'model': agent_foundation_models[0]
}

with open('mortgage_application_agent_strands.pkl', 'wb') as f:
    pickle.dump(agent_config, f)

print("Agent configuration saved successfully!")
%store mortgage_application_agent

PicklingError: Can't pickle <function get_mortgage_app_doc_status at 0x7feed5d23420>: it's not the same object as __main__.get_mortgage_app_doc_status